### Lab3

In [1]:
import cv2 as cv
import numpy as np

def to_screen(name, file):
    cv.imshow(name, file)  
    cv.waitKey(0)
    cv.destroyAllWindows()  

### exercise 1

In [2]:
cx = 960
cy = 540
f = 960
# f = 1920

matrix = np.array([[f, 0, cx], [0, f, cy], [0, 0, 1]])

### assignment 1

In [3]:
vertices = np.array([[0., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 1.],
       [1., 0., 0.],
       [1., 0., 1.],
       [1., 1., 0.],
       [1., 1., 1.]])

edges = np.array([[0,2],
                  [0,4],
                  [0,1],
                  [4,5],
                  [1,5],
                  [2,3],
                  [1,3],
                  [2,6],
                  [4,6],
                  [3,7],
                  [6,7],
                  [5,7],
                  ])

In [4]:
vertices = np.array([
    [0, 0, 0],  # 0
    [1, 0, 0],  # 1
    [1, 1, 0],  # 2
    [0, 1, 0],  # 3
    [0, 0, 1],  # 4
    [1, 0, 1],  # 5
    [1, 1, 1],  # 6
    [0, 1, 1]   # 7
])

vertices[:,2] += 5

edges = np.array([    [0, 1], [1, 2], [2, 3], [3, 0],  # bottom square
    [4, 5], [5, 6], [6, 7], [7, 4],  # top square
    [0, 4], [1, 5], [2, 6], [3, 7]   # vertical edges
])


In [5]:
# homogeneous coordinates of projected vertices
homogeneous_coords = vertices @ matrix.T

# 2D image coordinates of projected vertices
image_coords = homogeneous_coords[:, :2] / homogeneous_coords[:, 2:]
image_coords = np.round(image_coords).astype(int)
# image_coords

In [6]:
import cv2

# create empty 1080p image
image = np.zeros((1080, 1920, 3), dtype=np.uint8)

# draw vertices as green circles
for i in range(8):
    cv2.circle(image, tuple(image_coords[i]), 10, (0, 255, 0), -1)

# draw edges as red lines
for i, j in edges:
    cv2.line(image, tuple(image_coords[i]), tuple(image_coords[j]), (0, 0, 255), 2)

cv2.imshow("Cube", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Question 2: what happens if you double the focal length?

### Question 3

The reason why only even powers of r are included in this polynomial is because of symmetry. The radial distortion model assumes that the distortion is symmetric with respect to the principal point. 

Therefore, any odd power of r would introduce an asymmetry in the distortion model that does not exist in reality.

### Assignment 2

In [7]:
### source: https://learnopencv.com/camera-calibration-using-opencv/

import glob

# Define the calibration pattern size
pattern_size = (10, 6)

# Define the object points in 3D (assumes a flat calibration pattern)
objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

# Initialize arrays to store object points and image points from all images
objpoints = []
imgpoints = []

# Loop over calibration images and find chessboard corners
calibration_files = glob.glob("../calibration_frames/*.png")

for fname in calibration_files:
    # Load the image
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)

    # If corners found, add object points and image points to arrays
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw corners on image and display
        cv2.drawChessboardCorners(img, pattern_size, corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()




In [8]:
# Use the object and image points to estimate camera parameters
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Print the camera matrix and distortion coefficients
print("Intrinsic matrix:")
print(mtx)
print("Distortion coefficients:")
print(dist)

Intrinsic matrix:
[[583.72423649   0.         647.13947331]
 [  0.         588.20496961 338.38519019]
 [  0.           0.           1.        ]]
Distortion coefficients:
[[-2.41188339e-01  6.80055862e-02 -8.82713982e-04 -2.37905310e-05
  -9.40313880e-03]]


### Assignment 3 

In [35]:
from numpy import random
objpoints = np.array(objpoints)
imgpoints = np.array(imgpoints)

ret_arr = []
mtx_arr = []
dist_arr = []
rvecs_arr = []
tvecs_arr = []

for i in range(10):
    x = random.randint(len(objpoints), size=(20))
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints[x], imgpoints[x], gray.shape[::-1], None, None)
    ret_arr.append( ret)
    mtx_arr.append( mtx)
    dist_arr.append(dist)
    rvecs_arr.append(rvecs)
    tvecs_arr.append(tvecs)

533.7214764605827

In [36]:
# Extract the parameters cx, cy, f1, and f2 from each camera matrix
cx_list = [matrix[0, 2] for matrix in mtx_arr]
cy_list = [matrix[1, 2] for matrix in mtx_arr]
f1_list = [matrix[0, 0] for matrix in mtx_arr]
f2_list = [matrix[1, 1] for matrix in mtx_arr]

# Calculate the standard deviation of each parameter
cx_std = np.std(cx_list)
cy_std = np.std(cy_list)
f1_std = np.std(f1_list)
f2_std = np.std(f2_list)

print(cx_std)
print(cy_std)
print(f1_std)
print(f2_std)



39.71708460982649
37.80772413831836
1028.2624085892985
467.45226731212176


In [49]:
for i in range(0,len(mtx_arr)):
    img = cv.imread('../calibration_frames/img_0005.png')
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx_arr[i], dist_arr[i], (w,h), 1, (w,h))
    # undistort
    dst = cv.undistort(img, mtx_arr[i], dist_arr[i], None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imwrite(f"calib{i}result.png", dst)

## Exercise 3

### Assignment 5

In [55]:
shadow = cv.imread("../afbeeldingen/shadow.png")
copy_shadow = shadow.copy()
rows, cols = shadow.shape[:2]

m = -0.15
horizontal_shear_M = np.array([[1, m, 0], [0, 1, 0]], dtype="float32")

print(horizontal_shear_M.dtype)

# om naar rechts te verschuiven
horizontal_shear_M[0, 2] += 260


shear_trans = cv.warpAffine(copy_shadow, horizontal_shear_M, (2*cols, rows))

print(shear_trans.dtype)
to_screen("original_shadow", shadow)
to_screen("shear_trans", shear_trans)


float32
uint8


### Assignment 6